## Examples and applications

Now that we have gone through an overview of the all the parts that make up a variational algorithm and some instances, it is time to put them into practice.

Imagine that we want to use a variational algorithm to find the eigenvalues and eigenstates of the following observables:

$$
\hat{O}_1 = 
\begin{pmatrix} 
-1 & 0 & 0 & -5 \\
0 & 5 & 1 & 0 \\
0 & 1 & 5 & 0 \\
-5 & 0 & 0 & -1 \\
\end{pmatrix} = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

with eigenvalues

$$
\left\{
\begin{array}{c}
\lambda_0 = -6 \\
\lambda_1 = 4 \\
\lambda_2 = 4 \\
\lambda_3 = 6
\end{array}
\right\},
$$

and eigenstates

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
\end{array}
\right\};
$$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable_1 = SparsePauliOp.from_list([
    ("II", 2),
    ("XX", -2),
    ("YY", 3),
    ("ZZ", -3)
])

and

$$
\hat{O}_2 = 
\begin{pmatrix} 
-2 & 0 & 0 & -5 \\
0 & 6 & -1 & 0 \\
0 & -1 & 6 & 0 \\
-5 & 0 & 0 & -2 \\
\end{pmatrix} = 2 II - 3 XX + 2 YY - 4 ZZ ,
$$

with eigenvalues

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\},
$$

and eigenstates

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{array}
\right\}.
$$

In [ ]:
observable_2 = SparsePauliOp.from_list([
    ("II", 2),
    ("XX", -3),
    ("YY", 2),
    ("ZZ", -4)
])

For these examples we will use the following ansatz:

In [ ]:
from qiskit.circuit.library import TwoLocal

ansatz = TwoLocal(2, rotation_blocks=['rz', 'ry'], entanglement_blocks='cx', entanglement='linear', reps=1)

ansatz.decompose().draw("mpl")

So, in this case, the variational parameters will be $\vec\theta \equiv (\theta_0, \theta_1, \cdots, \theta_7)$, and the circuit 

$$
U_A(\vec{\theta}) = 
[RY(\theta_7)RZ(\theta_5)\otimes RY(\theta_6)RZ(\theta_4)] 
\cdot CNOT_{0,1} \cdot 
[RY(\theta_3)RZ(\theta_1)\otimes RY(\theta_2)RZ(\theta_0)]
$$

Notice that the reference state $|\rho\rangle = |0\rangle$, which means that $U_R = I$, and $U_A(\vec\theta) = U_V(\vec\theta)$. In other words, this ansatz is only made with a variational form.

## VQE

In this first example, you will run VQE to find the lowest eigenvalue of the first observable $\hat{O}_1$ and look at all the output results. In order to initialize a Qiskit [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) object, you need to provide an `Estimator` instance, be it Terra's, Runtime's or any other implementation. It is also necessary to include the [ansatz](ansatz.ipynb) circuit and a classical [optimizer](optimization.ipynb). 

In this case we will use, Terra's [`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator) and the optimizer will be [`SLSQP`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SLSQP.html) (i.e. Sequential Least SQuares Programming). Additionally, we will set the initial point for the `SLSQP` optimizer to $\vec\theta_0 = (1, \cdots, 1)$.

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

estimator = Estimator()
optimizer = SLSQP()
vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))

Now that we have initialized the `VQE` instance, we can get the results with the [`VQE.compute_minimum_eigenvalue`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue) method. Let us look at the results.

In [ ]:
result = vqe.compute_minimum_eigenvalue(observable_1)
print(result)

The [`VQEResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQEResult.html) we obtain explains the following:
* `aux_operators_evaluated`: the expected values of any auxiliar operators we might include.
* `cost_function_evals`: Number of cost function evaluations.
* `eigenvalue`: the resulting eigenvalue.
* `optimal_circuit`: the parametrized circuit from which the solution was found. To find the eigenstates you have to bind the optimal values to the parameters.
* `optimal_parameters`: The set of optimal parameters as a dictionary
* `optimal_point`: an array with the optimal parameter values
* `optimal_value`: the optimal value of the cost function.
* `optimizer_evals`: number of optimizer evaluations.
* `optimizer_result`: results of the optimizer as an [`OptimizerResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.OptimizerResult.html) object.
* `optimizer_time`: time used for optimization (in seconds).

Let us look at the `optimizer_result`:

In [ ]:
print(result.optimizer_result)

Here the parameters are:
* `fun`: the cost function value at the final step.
* `jac`: the final gradient of the minimization.
* `nfev`: number of cost function evaluations.
* `nit`: number of optimizer iterations.
* `njev`: number of gradient evaluations.
* `x`: parameter values at the final step.

Of all this information, however, the most important part is the eigenvalue. Let us compare it with the theoretical value:

In [ ]:
from numpy.linalg import eigvalsh

eigenvalues = eigvalsh(observable_1.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}")

As you can see, the result is extremely close to the ideal.

However, we still haven't looked at the eigenstates, since they were not part of `results`. For this purpose, let us bind the optimal parameter values `result.optimal_parameters` to `results.optimal_circuit` and define a [Statevector](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html) from that bound (i.e. non-parametrized) circuit.

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)
optimal_vector = Statevector.from_instruction(optimal_circuit)

rounded_optimal_vector = np.round(optimal_vector.data, 3)
print(f"EIGENSTATE: {rounded_optimal_vector}")

This result does not seem too close to the theoretical one of $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \equiv [\frac{1}{\sqrt{2}},0,0,\frac{1}{\sqrt{2}}]$. However, notice that eigenvectors are defined up to a constant factor. Furthermore, quantum states are always normalized and equivalent up to a global phase, so we can easily verify that these two statevectors are equivalent.

In [ ]:
from numpy.linalg import eigh

_, eigenvectors = eigh(observable_1.to_matrix())
min_eigenvector = eigenvectors.T[0]  # Note: transpose to extract by index

optimal_vector.equiv(min_eigenvector, atol=1e-4)

We can conclude that the state we obtained is equivalent to the ideal one up to $10^{-4}$.

### Add reference state

In the previous example we have not used any reference operator $U_R$. Now let us think about how the ideal eigenstate $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ can be obtained. Consider the following circuit.

In [ ]:
from qiskit import QuantumCircuit

ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0,1)

ideal_qc.draw("mpl")

We can quickly check that this circuit gives us the desired state.

In [ ]:
Statevector.from_instruction(ideal_qc)

Now that we have seen how a circuit preparing the solution state looks like, it seems reasonable to use a Hadamard gate as a reference circuit, so that the full ansatz becomes:

In [ ]:
reference = QuantumCircuit(2)
reference.h(0)
# Include barrier to separate reference from variational form
reference.barrier()

ref_ansatz = ansatz.decompose().compose(reference, front=True)

ref_ansatz.draw("mpl")

For this new circuit, the ideal solution could be reached with all the parameters set to $0$, so this confirms that the choice of reference circuit is reasonable.

Now let us compare the number of cost function evaluations, optimizer iterations and time taken with those of the previous attempt.

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
# You can change the ansatz of the already defined vqe object instead of creating a new one
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable_1)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")


How effective this is depends also on the choice of observable. Now let us solve the same problem as before but for the second observable $2 II - 3 XX + 2 YY - 4 ZZ$. As we have seen, the solution eigenstate is the same, except that the corresponding eigenvalue is now $-7$ instead of $-6$.

In [ ]:
# Reset the ansatz
vqe.ansatz = ansatz

result = vqe.compute_minimum_eigenvalue(observable_2)

Before comparing the results before and after the reference state, let us check whether the eigenvalue we obtain is close to the ideal one.

In [ ]:
eigenvalues = eigvalsh(observable_2.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}")

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable_2)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")

### Change initial point

Now that we have seen the effect of adding the reference state, we will go into what happens when we choose different initial points $\vec{\theta_0}$. In particular we will use $\vec{\theta_0}=(0,0,0,0,6,0,0,0)$ and $\vec{\theta_0}=(6,6,6,6,6,6,6,6,6)$. Remember that, as discussed when the reference state was introduced, the ideal solution would be found when all the parameters are $0$, so the first initial point should give fewer evaluations, iterations and time.

In [ ]:
vqe.initial_point = [0,0,0,0,6,0,0,0]

result = vqe.compute_minimum_eigenvalue(observable_1)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.initial_point = 6*np.ones(8)

result = vqe.compute_minimum_eigenvalue(observable_1)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

## VQD example

Now instead of looking for only the lowest eigenvalue of our observables, we will look for all $4$. Following the notation from the previous chapter (as well as that from Qiskit's [VQD](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.html) class), that means $k=4$.

Remember that the cost functions of VQD are:

$$
C_{l}(\vec{\theta}) := 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{l-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2 
\quad \forall l\in\{1,\cdots,k\}=\{1,\cdots,4\}
$$

This is particularly important because a vector $\vec{\beta}=(\beta_0,\cdots,\beta_{k-1})$ (in this case $(\beta_0, \beta_1, \beta_2, \beta_3)$) must be passed as an argument when we define the `VQD` object.

Also, in Qiskit's implementation of VQD, instead of considering the effective observables described in the previous notebook, the fidelities $|\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2$ are calculated directly via the [`ComputeUncompute`](https://qiskit.org/documentation/stubs/qiskit.algorithms.state_fidelities.ComputeUncompute.html) algorithm, that leverages a `Sampler` primitive to sample the probability of obtaining $|0\rangle$ for the circuit
$U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})$. This works precisely because this probability is

$$
\begin{aligned}

p_0

& = |\langle 0|U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})|0\rangle|^2 \\[1mm]

& = |\big(\langle 0|U_A^\dagger(\vec{\theta})\big)\big(U_A(\vec{\theta^j})|0\rangle\big)|^2 \\[1mm]

& = |\langle \psi(\vec{\theta}) |\psi(\vec{\theta^j}) \rangle|^2 \\[1mm]

\end{aligned}
$$


Finally, to try out a new optimizer, let us use [`COBYLA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.COBYLA.html) instead instead of `SLSQP`.

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

optimizer = COBYLA()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

k = 4
betas = [40, 60, 30, 30]

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)

As the only API difference from the previous examples, notice that instead of calling the `VQE.compute_minimum_eigenvalue` method, we will call [`VQD.compute_eigenvalues`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.compute_eigenvalues.html).

This time we will examine first the second observable, $\hat{O}_2 := 2 II - 3 XX + 2 YY - 4 ZZ$. The reasoning behind this will become apparent later.

In [ ]:
result = vqd.compute_eigenvalues(observable_2)
print(result)

The [VQDResult](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQDResult.html) we obtain is completely analogous to the `VQEResult`. It only differs in that each attribute is a list whose $i$-th element corresponds to the $i$-th eigenvalue.

Now that we have seen the eigenvalues, let us compare the experimental eigenvectors with the theoretical ones:

In [ ]:
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector.from_instruction(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

These results are the same as the expected ones except from a small approximation error and global phase.

Now let us solve this problem for the first observable $\hat{O}_1 := 2 II - 2 XX + 3 YY - 3 ZZ$.

In [ ]:
result = vqd.compute_eigenvalues(observable_1)

## PRINT
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector.from_instruction(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Here the eigenstates corresponding to $\lambda_1 = \lambda_2 = 4$ are not the same as the expected ones: $|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$ and $|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$. This happens precisely because as $\lambda_1=\lambda_2$, any complex linear combination of those $\alpha_1 |\phi_1\rangle + \alpha_2 |\phi_2\rangle = \frac{1}{\sqrt{2}}(\alpha_1 |00\rangle + \alpha_2 |01\rangle - \alpha_2 |10\rangle - \alpha_1 |11\rangle)\equiv \frac{1}{\sqrt{2}}[\alpha_1, \alpha_2, -\alpha_2, -\alpha_1]$ is also a eigenstate with the same eigenvalue. That is exactly what we are seeing with these results.

## Change betas

As mentioned in the instances chapter, the values of $\vec{\beta}$ should be bigger than the difference between eigenvalues. Let us see what happens when they do not satisfy that condition.

In [ ]:
vqd.betas = np.ones(4)
result = vqd.compute_eigenvalues(observable_2)

## PRINT
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector.from_instruction(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

This time, the optimizer returns the same state $|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ as a proposed solution to all eigenstates: which is clearly wrong. This happens because the betas were too small to penalize the minimum eigenstate in the successive cost functions; therefore, it was not excluded from the effective search space in later iterations of the algorithm, and always chosen as the best possible solution.

## Variational algorithms with Qiskit Runtime

In [ ]:
## I'm following this how-to guide https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/noisy_simulators.html
## the idea is to use runtime with noisy simulator.


from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
service.backends()

In [ ]:
# use the simulator

backend = service.backend("ibmq_qasm_simulator")

In [ ]:
from qiskit.providers.fake_provider import FakeManilaV2
from qiskit_aer.noise import NoiseModel

#create noise model from the backend

fake_backend = FakeManilaV2()
noise_model = NoiseModel.from_backend(fake_backend)

In [ ]:
from qiskit_ibm_runtime import Options

options = Options()
print(options)

In [ ]:
# set options to use the noise model

options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.operation_names,
    "coupling_map": list(fake_backend.coupling_map),
    "seed_simulator": 42
}

# had to refactor the coupling map as a list because otherwise I get error.

options.optimization_level=0
options.resilience_level=0

In [ ]:
from qiskit_ibm_runtime import Estimator as RuntimeEstimator
from qiskit_ibm_runtime import Session

# Run VQE with the simulator. Using noise model and specifications from ManilaV2
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

print(result)

In [ ]:
from qiskit.quantum_info import Statevector
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# Now I set optimization level from 0 to 3

options.optimization_level=3

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

#results are worse?
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# now increase resilience from 0 to 2
# if I try setting it to 3 I get a big error

# TLDR: optimization level 3 and resilience level 2.

options.resilience_level=2

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

#results improved but very close to the noisy ones without mitigation
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# Increase shots from 4000 to 15000

options.execution.shots = 15000

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

# Results are actually worse
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# the state seemed fine at first sight even though the eigenvalue given by estimator was not.
# I'll compare the result of estimator for the optimal statevector for the noisy and noiseless estimators.

# Noisy gives a very bad result

with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    resultado=estimator.run(opt_qc, observable_1).result()
    session.close()

print(resultado)

In [ ]:
# noiseless gives a reasonable one (close to the ideal of -6) for the exact same state

with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session)
    resultado=estimator.run(opt_qc, observable_1).result()
    session.close()

print(resultado)

----------------------

-----------------

In [ ]:
## Here I repeat everything for FakeManila instead of FakeManilaV2
## following this how-to guide https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/noisy_simulators.html

In [ ]:
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel

fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

In [ ]:
from qiskit_ibm_runtime import Options

options = Options()
print(options)

In [ ]:
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42
}

options.optimization_level=0
options.resilience_level=0

In [ ]:
from qiskit_ibm_runtime import Estimator as RuntimeEstimator
from qiskit_ibm_runtime import Session

with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

print(result)

In [ ]:
from qiskit.quantum_info import Statevector
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# increase optimization level from 0 to 3

options.optimization_level=3

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

# the result now is a lot better than when I did the same for V2. Why?
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# increase resilience

options.resilience_level=2

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

# the result now is a lot worse!
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# see what happens when I set resilience level to 3

options.resilience_level=3

In [ ]:
# this part is exactly the same again

with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# increase shots

options.execution.shots = 15000

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()


# result even worse
print(result)

In [ ]:
opt_qc = result.optimal_circuit.bind_parameters(result.optimal_parameters)

opt_vector = Statevector.from_instruction(opt_qc)

print(f"The lowest eigenvalue obtained with the noisy backend is {result.eigenvalue} and its statevector is {opt_vector}.\n The ideal eigenvalue is {-6} and its eigenvector is {[1,0,0,1]/np.sqrt(2)}")

In [ ]:
# lower optimization level to 0

options.optimization_level=0

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

# not a big difference. A bit worse
print(result)

In [ ]:
# lower resilience from 2 to 1

options.resilience_level=1

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session, options=options)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))
    result=vqe.compute_minimum_eigenvalue(observable_1)
    session.close()

# why is it better now?
print(result)

----------------------

-----------------

In [ ]:
from qiskit_ibm_runtime import Sampler as RuntimeSampler

with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session)
    
    sampler = RuntimeSampler(session=session)
    fidelity = ComputeUncompute(sampler)


    vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)
    result=vqd.compute_eigenvalues(observable_2)

# An eigenvalue of 16??? The highest eigenvalue for that observable was 7. It should be completely impossible to get 16

print(result)

In [ ]:
estimator.options

In [ ]:
estimator.options.execution["shots"]